# USB Data
----

## Understanding USB Data in the Context of Insider Threat
**What are the features/properties of USB data that will inform modeling for unsual behaviour?**

<img src="usb-insider.png" alt="Haystax's Data Science Workflow" width="25%" />
<figcaption>Image credit: www.secudrives.com.</figcaption>

USB data shows the connect and disconnect activity of employees with corresponding attributes such as `date`, `user`, `pc`, and `file_tree`.

### Initial thoughts

* Users are typically assigned a normal/average number of thumb drive uses per day. Deviations from a user's normal usage can be considered significant.
* The `file_tree` field is a semicolon-delimited list of directories on the device. We can look at these directories for some insights.
* Malicious insiders will often copy large amounts of proprietary data to a USB device  or a personal device
* Unauthorized downloading of data onto a personal USB drive or other storage medium.
* Copying data to an unsecured USB is an example of insider threat that cannot be detected with controls such as Firewalls, IDS, IPS, SIEM because they are focused on external attacks.  

* Understanding the context: 
  * With external attacks, the line between good and bad activity is very clear. An external entity should be accessing your web page, for example, but should never be running software that tries to log in with 500 different passwords in a short time. Therefore, typical preventive measures are very effective. With an insider threat, a user performs a normal set of actions, but the context in which the user executes them is quite different. As an example, one user could be accessing files for project A because she is actively working with the client. This is quite different than the user who is accessing files for project B, a project he has not worked on for several months, then immediately copies the information to a USB drive. In both cases, the access is similar, but the context makes these very different situations.

* Copying, modifying and deleting data on a USB during offboarding is a potential indicator of insider threat.
* An insider visits a restricted site (e.g. wikileaks) after having inserted a USB device
* An example of a malicious scenario from CERT:
  * User who did not previously use removable drives or work after hours begins logging in after hours, using a removable drive, and uploading data to wikileaks.org. Leaves the organization shortly thereafter.

First, let's set up our working environment including required libraries.

In [1]:
# %load modeling.py
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import dask.dataframe as dd
import pickle
import seaborn as sns             # for static plots
import matplotlib.pyplot as plt
from bqplot import *              # for interactive plots
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as ipw
fig_layout = ipw.Layout(width = "100%", height = "600px")
sns.set_context('notebook', font_scale = 1.1)
np.random.seed(12345)
rc = {'xtick.labelsize': 40, 'ytick.labelsize': 40, 'axes.labelsize': 40, 'font.size': 40,
      'lines.linewidth': 4.0, 'lines.markersize': 40, 'font.family': "serif", 'font.serif': "cm",
      'savefig.dpi': 200, 'text.usetex': False, 'legend.fontsize': 40.0, 'axes.titlesize': 40, 
      "figure.figsize": [24, 16]}
sns.set(rc = rc)
sns.set_style("ticks")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from warnings import filterwarnings
filterwarnings("ignore")

ValueError: You are tring to use the old GPU back-end. It was removed from Theano. Use device=cuda* now. See https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29 for more information.

### Import the standardized data
Import the standardized data and peform sanity checks to verify its consistency with the Haystax's core-data dictionary including:
* required columns, and 
* correct data types.

In [2]:
# First, specify data source here
hard_disk = "/opt/usb/"
folder = "cert/standardized/usb"
file_type = "*.parquet" # may include csv, hdf, json, text, etc
data_path = os.path.join(hard_disk, folder, file_type)

In [3]:
def import_data (path):
    """Import data from the source into a distributed dask dataframe.  
    Then check whether data is standardized to Haystax's core-data-dictionary.
    Then check whether the data types are correct i.e categorical, int, datetime, string 
    """
    try:
        ddf = dd.read_parquet(path)
    except IOError:
        print("Cannot find this data source: "+hard_disk+folder)
        print("Contact David Jones for the correct data source")
        print("Quiting!")
        quit()


    # Check if data is standardized to Haystax's core data dictionary    
    required_columns = ["record_id", "employee_id", "timestamp", "computer_id", 
                        "file_tree", "connect_activity"]
    try:
        if not (ddf.columns == required_columns).all():
            raise ValueError
    except:
        print("The columns in the data are not standardized.")
        print("Quiting!")
        print("Contact David Jones for Haystax's standard-core-data dictionary")
        quit() 
        
    

     
        #     # Check if the columns are of the right data type i.e categorical, datetime, int, or string    
#     required_data_types = ["category", "category", "category",
#                         "str", "datetime64[ns]", "int64", 
#                         "float", "str", "datetime64[ns]"]
    
#     data_types = [ddf.record_id.dtype == 'category', 
#                   ddf.sender_employee_id.dtype == 'category',
#                   ddf.sender_username.dtype == 'category',
#                   ddf.subject.dtype == 'str',
#                   ddf.timestamp.dtype == 'datetime64[ns]',
#                   ddf.number_of_attachments.dtype == 'int64',
#                   ddf.attachment_size.dtype == 'float',
#                   ddf.email_text.dtype == 'str',
#                   ddf.file_date.dtype == 'datetime64[ns]']
#     data_types = [True, True, True, False, True, False, False, False, False]
    
#     try:
#         if not (data_types == required_columns).all():
#             raise ValueError
#     except:
#         print("The columns in the data are not of the correct data type.")
#         print("Quiting!")
#         print("Contact David Jones for Haystax's standard-core-data dictionary")
#         quit() 

    return ddf

In [4]:
ddf = import_data(path = data_path)
ddf.head().info()
ddf.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
record_id           5 non-null object
employee_id         5 non-null object
timestamp           5 non-null datetime64[ns]
computer_id         5 non-null object
file_tree           5 non-null object
connect_activity    5 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 320.0+ bytes


Index(['record_id', 'employee_id', 'timestamp', 'computer_id', 'file_tree',
       'connect_activity'],
      dtype='object')

In [5]:
df = ddf.get_partition(n = 1).compute()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711646 entries, 0 to 711645
Data columns (total 6 columns):
record_id           711646 non-null object
employee_id         711646 non-null object
timestamp           711646 non-null datetime64[ns]
computer_id         711646 non-null object
file_tree           711646 non-null object
connect_activity    711646 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 32.6+ MB


In [6]:
# df.to_csv(os.path.join(hard_disk, folder, "usb_filtered.csv"))
ddf.npartitions
ddf.divisions

3

(None, None, None, None)

In [7]:
# df = pd.read_csv(os.path.join(hard_disk, folder, "usb_filtered.csv"))
df.head(n = 6)

,record_id,employee_id,timestamp,computer_id,file_tree,connect_activity
0,{O8W0-V9IB85HG-1450TSUE},CQS2883,2010-08-16 10:58:11,PC-6220,nan,Disconnect
1,{J0G8-V7JL47TS-2815LUVW},WNW2002,2010-08-16 10:58:30,PC-6583,nan,Disconnect
2,{H3G1-Y0TF18ZX-6825OMMX},ESS3318,2010-08-16 10:58:51,PC-2607,R:\;R:\83HcQ51;R:\994GYK8;R:\ESS3318,Connect
3,{F3W8-C5DT43NQ-4939NPGL},REJ2666,2010-08-16 10:58:55,PC-9016,R:\;R:\39d26d6;R:\44V73d6;R:\81R64j6;R:\85VB4T...,Connect
4,{I9M2-O1MV98PV-2407PPSH},GNT0221,2010-08-16 10:58:59,PC-6427,nan,Disconnect
5,{Z6A0-B6MN65EU-9724FMXV},BDJ3001,2010-08-16 10:59:04,PC-4401,R:\;R:\43WTgN0;R:\87rRp41;R:\BDJ3001,Connect


### Step 1: Prepare the data

In [ ]:
# create some data as a Numpy array or pandas df
data = np.random.randn(40000)
#y = pm.Minibatch(data, batch_size=500)
y = theano.shared(data)
y.get_value()

## What is the proportion of connect activity?

Some important notes: 
* Users are assigned a normal/average number of thumb drive uses per day. Deviations from a user's normal usage can be considered significant.
* Some connect events may be missing disconnect events, because users can power down machine before removing drive

In [8]:
connect_act = df.groupby("connect_activity").connect_activity.count()
connect_act

connect_activity
Connect       357017
Disconnect    354629
Name: connect_activity, dtype: int64

In [9]:
xs = OrdinalScale()
ys = LinearScale()
x = connect_act.index
y = connect_act.values/1e3

bar_chart = Bars(x = x, y = y, 
                 scales = {'x': xs, 'y': ys}, 
                 colors = ["limegreen"], 
                 padding = 0.2, 
                 display_legend = False, 
                 labels = ["Connect Activity"])

tt = Tooltip(fields = ["x", "y"], 
             formats = ["", "0.2f"], 
             labels = ["Activity", "Count"])

bar_chart.tooltip = tt
bar_chart.selected_style = {"fill" : "red"}
bar_chart.interactions = {
    'legend_hover': 'highlight_axes',
    'hover': 'tooltip', 
    'click': 'select',
}

x_ax = Axis(scale=xs, label='Connect Activity', grid_lines='solid')
y_ax = Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='Count', grid_lines='solid')


panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})

fig1 = Figure(marks=[bar_chart], 
              axes=[x_ax, y_ax], 
              animation_duration=5000, 
              layout = fig_layout, 
              background_style={'fill': 'aliceblue'}, 
              title = "Proportion of Connect Activity", 
              legend_location = "top-right", 
              legend_style = {"fill" : "white"}
             )
    

ipw.VBox([fig1])

There are always more connect activities than disconnect activities. This implies that sometimes insiders power off their computers before disconneting their USB devices.

## Which computer IDs experience the most activity?

In [10]:
pc = (df.groupby(["computer_id", "connect_activity"])["employee_id"]
      .size()
      .unstack()
      .sort_values(by = "Connect", ascending = False)
      .head(n = 10)
     )
pc

connect_activity,Connect,Disconnect
computer_id,,
PC-3471,2048,2042
PC-1105,2036,2026
PC-0937,2029,2014
PC-3154,2009,1998
PC-2158,2000,1987
PC-7117,1991,1983
PC-2659,1978,1965
PC-2057,1963,1949
PC-5849,1959,1947


In [11]:
xs = OrdinalScale()
ys = LinearScale()
x = pc.index
y = [pc.Connect/1e3, pc.Disconnect/1e3]

bar_chart = Bars(x = x, y = y, 
                 scales = {'x': xs, 'y': ys}, type = "grouped",
                 colors = ["hotpink", "orange"], 
                 padding = 0.1,
                 display_legend = True,
                 labels = ["Connect", "Disconnect"])

tt = Tooltip(fields = ["x", "y"], 
             formats = ["", "0.1f"], 
             labels = ["Computer ID", "Activity"])

bar_chart.tooltip = tt
bar_chart.selected_style = {"fill" : "red"}
bar_chart.interactions = {
    'legend_hover': 'highlight_axes',
    'hover': 'tooltip', 
    'click': 'select',
}

x_ax = Axis(scale=xs, label='Computer ID', grid_lines='solid')
y_ax = Axis(scale=ys, orientation='vertical', tick_format='0.1f', label='Count', grid_lines='solid')


panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})

fig2 = Figure(marks=[bar_chart], 
              axes=[x_ax, y_ax], 
              animation_duration=5000, 
              layout = fig_layout, 
              background_style={'fill': 'aliceblue'}, 
              title = "Proportion of Connect & Disconnet Activity", 
              legend_location = "top-right", 
              legend_style = {"fill" : "white"}
             )
    

ipw.VBox([fig2])

There are always more connect activities than disconnect activities. This implies that sometimes insiders power off their computers before disconneting their USB devices.

## What file directories are most common?

In [12]:
df["file_tree2"] = df.file_tree.apply(lambda x: x.split(";"))

In [13]:
file_tree2 = df.file_tree2
combined_list=[]
for x in file_tree2:
    combined_list.extend(x) 

In [14]:
print("There are {count} unique directories".format(count = len(set(combined_list))))
print("There are {count} directories".format(count = len(combined_list)))
print("The directory 'R:\\39d26d6' appears {count} times".format(count = combined_list.count("R:\\39d26d6")))

There are 2275 unique directories
There are 1807088 directories
The directory 'R:\39d26d6' appears 1492 times


In [15]:
dir_count = pd.Series(combined_list).value_counts()
dir_count.head(n = 10)

R:\           357017
nan           354629
R:\13jd423      2046
R:\CJM0273      2046
R:\49FP5C2      2046
R:\51n6647      2046
R:\JDM1042      2033
R:\84xF4F2      2033
R:\AKB2312      2025
R:\787PQ59      2006
dtype: int64

In [16]:
xs = OrdinalScale()
ys = LinearScale()
x = dir_count.tail(n = 10).index
y = dir_count.tail(n = 10).values

bar_chart = Bars(x = x, y = y, 
                 scales = {'x': xs, 'y': ys}, 
                 colors = ["dodgerblue"], 
                 padding = 0.2, 
                 display_legend = False, 
                 labels = ["Directory name"])

tt = Tooltip(fields = ["x", "y"], 
             formats = ["", "f"], 
             labels = ["Directory", "Count"])

bar_chart.tooltip = tt
bar_chart.selected_style = {"fill" : "red"}
bar_chart.interactions = {
    'legend_hover': 'highlight_axes',
    'hover': 'tooltip', 
    'click': 'select',
}

x_ax = Axis(scale=xs, label='Directory name', grid_lines='solid')
y_ax = Axis(scale=ys, orientation='vertical', tick_format='f', label='Count (log)', grid_lines='solid')


panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})

fig3 = Figure(marks=[bar_chart], 
              axes=[x_ax, y_ax], 
              animation_duration=5000, 
              layout = fig_layout, 
              background_style={'fill': 'aliceblue'}, 
              title = "Proportion of Directory Access", 
              legend_location = "top-right", 
              legend_style = {"fill" : "white"}
             )
    

ipw.VBox([fig3])

In [17]:
_dir = (df[df["file_tree"] != "nan"]
        .groupby("connect_activity")
       )
_dir.connect_activity.count()

connect_activity
Connect    357017
Name: connect_activity, dtype: int64

There are 0 directories associated with disconnect activity.

In [18]:
_dir.head()

,record_id,employee_id,timestamp,computer_id,file_tree,connect_activity,file_tree2
2,{H3G1-Y0TF18ZX-6825OMMX},ESS3318,2010-08-16 10:58:51,PC-2607,R:\;R:\83HcQ51;R:\994GYK8;R:\ESS3318,Connect,"[R:\, R:\83HcQ51, R:\994GYK8, R:\ESS3318]"
3,{F3W8-C5DT43NQ-4939NPGL},REJ2666,2010-08-16 10:58:55,PC-9016,R:\;R:\39d26d6;R:\44V73d6;R:\81R64j6;R:\85VB4T...,Connect,"[R:\, R:\39d26d6, R:\44V73d6, R:\81R64j6, R:\8..."
5,{Z6A0-B6MN65EU-9724FMXV},BDJ3001,2010-08-16 10:59:04,PC-4401,R:\;R:\43WTgN0;R:\87rRp41;R:\BDJ3001,Connect,"[R:\, R:\43WTgN0, R:\87rRp41, R:\BDJ3001]"
6,{J2U3-J6MT33ZB-3858WFSX},CCF3529,2010-08-16 10:59:06,PC-1616,R:\;R:\CCF3529,Connect,"[R:\, R:\CCF3529]"
7,{N5C0-N1UH90WF-6176BDOP},TGP1343,2010-08-16 10:59:08,PC-6751,R:\;R:\21JD983;R:\38q7Mn4;R:\TGP1343,Connect,"[R:\, R:\21JD983, R:\38q7Mn4, R:\TGP1343]"


## What periods experience the most activity?

In [19]:
connect_time = (df[df["connect_activity"] == "Connect"].groupby(df.timestamp.dt.hour)
                .connect_activity
                .count()
#                 .sort_values(ascending = False)
                .reset_index()
)
connect_time.timestamp = pd.to_datetime(connect_time.timestamp, unit = 'h')

In [20]:
date_fmt = "%H"
xs = DateScale(date_format = date_fmt)
ys = LinearScale()
x = connect_time.timestamp
y = connect_time.connect_activity/1e3

bar_chart = Bars(x = x, y = y, 
                 scales = {'x': xs, 'y': ys}, 
                 colors = ["limegreen"],
                 display_legend = False, 
                 labels = ["Number of USB Connections"]
                )

tt = Tooltip(fields = ["x", "y"], 
             formats = ["", "f"], 
             labels = ["Date", "Number of USB Connections"]
            )

bar_chart.tooltip = tt
bar_chart.selected_style = {"fill" : "red"}
bar_chart.interactions = {
    'legend_hover': 'highlight_axes',
    'hover': 'tooltip', 
    'click': 'select',
}

x_ax = Axis(scale=xs, label='Hours of the Day', grid_lines='solid')
y_ax = Axis(scale=ys, orientation='vertical', tick_format='f', 
            label='Number of USB Connections', grid_lines='solid')


panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})

fig4 = Figure(marks=[bar_chart], 
              axes=[x_ax, y_ax], 
              animation_duration=5000, 
              layout = fig_layout, 
              background_style={'fill': 'aliceblue'}, 
              title = "USB Connections", 
              legend_location = "top-right"
             )
    

ipw.VBox([fig4])

In [21]:
connect_time_per_person = (df[df["connect_activity"] == "Connect"]
                           .groupby(["employee_id", df.timestamp.dt.hour])
                           .connect_activity
                           .count()
#                            .sort_values(ascending = False)
                           .reset_index()
            )
connect_time_per_person.timestamp = pd.to_datetime(connect_time_per_person.timestamp, unit = 'h')

## What are the connect periods per employee?

In [22]:
date_fmt = "%H"
xs = DateScale(date_format = date_fmt)
ys = LinearScale()
x = connect_time_per_person[connect_time_per_person["employee_id"] == "AAP1919"].timestamp
y = connect_time_per_person[connect_time_per_person["employee_id"] == "AAP1919"].connect_activity

bar_chart = Bars(x = x, y = y, 
                 scales = {'x': xs, 'y': ys}, 
                 colors = ["skyblue"],
                 display_legend = False, 
                 labels = ["Number of USB Connections"]
                )

tt = Tooltip(fields = ["x", "y"], 
             formats = ["", "f"], 
             labels = ["Date", "Number of USB Connections"]
            )

bar_chart.tooltip = tt
bar_chart.selected_style = {"fill" : "red"}
bar_chart.interactions = {
    'legend_hover': 'highlight_axes',
    'hover': 'tooltip', 
    'click': 'select',
}

x_ax = Axis(scale=xs, label='Hours of the Day', grid_lines='solid')
y_ax = Axis(scale=ys, orientation='vertical', tick_format='f', 
            label='Number of USB Connections', grid_lines='solid')


panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})

fig4 = Figure(marks=[bar_chart], 
              axes=[x_ax, y_ax], 
              animation_duration=5000, 
              layout = fig_layout, 
              background_style={'fill': 'aliceblue'}, 
              title = "USB Connections Per Employee", 
              legend_location = "top-right"
             )
    

ipw.VBox([fig4])

## What is the connection time per person?

In [23]:
date_fmt = "%H"
xs = DateScale(date_format = date_fmt)
ys = LinearScale()
x = connect_time_per_person[connect_time_per_person["employee_id"] == "AAC0610"].timestamp
y = connect_time_per_person[connect_time_per_person["employee_id"] == "AAC0610"].connect_activity

bar_chart = Bars(x = x, y = y, 
                 scales = {'x': xs, 'y': ys}, 
                 colors = ["skyblue"],
                 display_legend = False, 
                 labels = ["Number of USB Connections"]
                )

tt = Tooltip(fields = ["x", "y"], 
             formats = ["", "f"], 
             labels = ["Date", "Number of USB Connections"]
            )

bar_chart.tooltip = tt
bar_chart.selected_style = {"fill" : "red"}
bar_chart.interactions = {
    'legend_hover': 'highlight_axes',
    'hover': 'tooltip', 
    'click': 'select',
}

x_ax = Axis(scale=xs, label='Hours of the Day', grid_lines='solid')
y_ax = Axis(scale=ys, orientation='vertical', tick_format='f', 
            label='Number of USB Connections', grid_lines='solid')


panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})

fig4 = Figure(marks=[bar_chart], 
              axes=[x_ax, y_ax], 
              animation_duration=3000, 
              layout = fig_layout, 
              background_style={'fill': 'aliceblue'}, 
              title = "USB Connections Per Employee", 
              legend_location = "top-right"
             )

employee_dd = ipw.Dropdown(options=['AAC0610', 'AAP1919'], 
                              description = "Select Employee ID", 
                              style = {'description_width': 'initial'}
                              )   
def dd_changed(change):
    if employee_dd.value == 'AAC0610':
        bar_chart.y = (connect_time_per_person[connect_time_per_person["employee_id"] == "AAC0610"]
                       .connect_activity)
    else:
        bar_chart.y = (connect_time_per_person[connect_time_per_person["employee_id"] == "AAP1919"]
                       .connect_activity)
    
employee_dd.observe(dd_changed, 'value')

ipw.VBox([ipw.HBox([employee_dd]), fig4]) 